In [2]:
import numpy as np
import pandas as pd
import seaborn as sbn
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor as KNR
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error,make_scorer
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn import metrics
from math import log

sbn.set()

def get_diff_month(d1,d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month




In [110]:
df = pd.read_csv('./data/properties_2016.csv',low_memory=False)
original = df.copy()

train = pd.read_csv('./data/train_2016_v2.csv')

In [ ]:
# print len(df) #already checked for duplicate rows, the file doesn't have any

In [ ]:
print df.keys()

In [ ]:
# for d in df:
#     print df[d].value_counts(dropna = False)
print original['airconditioningtypeid'].describe()
print original['airconditioningtypeid'].value_counts(dropna=False)

In [ ]:
# print len(df)
# for d in original:
#     print d, original[d].isnull().sum()
# print merged['garagetotalsqft'].value_counts()#,merged['unitcnt'].value_counts()
# pd.merge(original,train,on = 'parcelid')['garagetotalsqft'].value_counts()

In [ ]:
missing_data = original.isnull().sum()
missing_data.sort_values(0,ascending=False,inplace = True)
missing_data = missing_data/(len(original))*100


# missing_data
missing_plot = missing_data.plot.bar(figsize= (24,14))
plt.title('Percentage of missing values vs. properties in data')
plt.xlabel('Properties of data')
plt.ylabel('Percentage of missing values')
plt.savefig('percent.pdf')
plt.show()
del missing_plot
del missing_data

We see that some variables have more than 99% of their data missing, using these variables for training might add noise to the data and hence adversely affect the training process 

In [ ]:
baseline_backup = original.copy()

baseline_backup['yardbuildingsqft17'].fillna(0, inplace = True)
baseline_backup['unitcnt'].fillna(1, inplace = True)
baseline_backup['garagetotalsqft'].fillna(0, inplace = True)
baseline_backup['fireplacecnt'].fillna(0, inplace = True)
baseline_backup['yearbuilt'].fillna(0, inplace = True)
baseline_backup['buildingqualitytypeid'].fillna(0, inplace = True)
baseline_backup['bedroomcnt'].fillna(0, inplace = True)
baseline_backup['lotsizesquarefeet'].fillna(0, inplace = True)
baseline_backup['poolcnt'].fillna(0,inplace = True)
baseline_backup['calculatedbathnbr'].fillna(0,inplace = True)
baseline_backup['airconditioningtypeid'].fillna(0,inplace=True)

#we cant keep object type data to use linear regression, so for the basline model, we drop those columns
baseline_backup.drop(['architecturalstyletypeid','basementsqft', 'buildingclasstypeid', 'decktypeid', 'garagecarcnt',
        'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'roomcnt', 'yardbuildingsqft26', 
         'numberofstories', 'fireplaceflag', 'fullbathcnt', 'heatingorsystemtypeid','storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid', 
         'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt', 
         'taxamount', 'bathroomcnt','hashottuborspa','propertycountylandusecode','propertyzoningdesc','taxdelinquencyflag'], axis=1, inplace=True)

In [ ]:
base_merge = pd.merge(baseline_backup,train,on = 'parcelid')

dates = []
month = []
for i in base_merge['transactiondate']:
    dates.append(get_diff_month(datetime.strptime(i,'%Y-%m-%d'),datetime(2015,1,1)))

base_merge['transactiondate'] = dates

base_merge.fillna(0, inplace=True)  #fill all missing values with 0 in the dataframe


base_logerror = base_merge['logerror']
# invlogerror = [10**i for i in logerror]
base_X_train, base_X_test, base_y_train, base_y_test = train_test_split(base_merge.drop(['logerror','parcelid'],axis=1), base_logerror, test_size=0.2, random_state=42)

In [ ]:
base_clf = LinearRegression(n_jobs=-1)
base_clf.fit(base_X_train,base_y_train)

pred = base_clf.predict(base_X_test)
# res = [log(i,10) if i>0 else -log(i*-1,10) for i in res]
print mean_absolute_error(base_y_test,pred), mean_squared_error(base_y_test,pred),base_clf.score(base_X_test,base_y_test)

The MSE is 0.0268 for the baseline model, in which we fill missing values in some of the columns, convert the transaction date to  a datetime variable  and use simple linear regression

In [ ]:
del base_X_train
del base_X_test
del base_y_train
del base_y_test

In [ ]:
df = original.copy()

In [111]:
df['yardbuildingsqft17'].fillna(0, inplace = True)
df['unitcnt'].fillna(1, inplace = True)
df['garagetotalsqft'].fillna(0, inplace = True)
df['fireplacecnt'].fillna(0, inplace = True)
df['hashottuborspa'].fillna(0, inplace = True)
df['yearbuilt'].fillna(df['yearbuilt'].median(), inplace = True)
df['buildingqualitytypeid'].fillna(7, inplace = True) #df['buildingqualitytypeid'].mode() results to 7
df['bedroomcnt'].fillna(df['bedroomcnt'].median(), inplace = True)
df['lotsizesquarefeet'].fillna(df['lotsizesquarefeet'].median(), inplace = True)
df['poolcnt'].fillna(0,inplace = True)
df['calculatedbathnbr'].fillna(df['calculatedbathnbr'].median(),inplace = True)
df['airconditioningtypeid'].fillna(-1,inplace=True)  #earlier it was replaced with 0,now replacing it with -1
df['propertylandusetypeid'].fillna(df['propertylandusetypeid'].mode(),inplace=True)
df['regionidcity'].fillna(df['regionidcity'].mean(),inplace=True) #mean is meaningless, but still

df['blockgroup'] = [i%10000 for i in df['rawcensustractandblock']] ##this is somewhat how the data looks
df['taxdelinquencyflag'].fillna(0,inplace=True)
df['taxdelinquencyflag'] =  df['taxdelinquencyflag'].map({'Y':1,0:0})
df['garagecarcnt'].fillna(0,inplace=True)
df['taxdelinquencyyear'] = pd.Series([15-i if i<16 else 115-i for i in df['taxdelinquencyyear']]).fillna(-1)

df['heatingorsystemtypeid'].fillna(2,inplace=True) # df['heatingorsystemtypeid'].mode() results in 2


df['actual_area'] = df[['finishedfloor1squarefeet','calculatedfinishedsquarefeet','finishedsquarefeet12', 'finishedsquarefeet13',
                        'finishedsquarefeet15', 'finishedsquarefeet50', 'finishedsquarefeet6']].max(axis=1)

###doing this for now, change later if time permits

df['fips'].fillna(df['fips'].median(),inplace = True)
df['latitude'].fillna(df['latitude'].median(),inplace = True)
df['longitude'].fillna(df['longitude'].median(),inplace = True)
df['regionidcounty'].fillna(df['regionidcounty'].median(),inplace = True)
df['regionidzip'].fillna(df['regionidzip'].median(),inplace = True)

df['actual_area'].replace(to_replace=1.0,value=np.nan,inplace=True)
df['actual_area'].fillna(df['actual_area'].median(),inplace=True)

####

df['hashottuborspa'] = df['hashottuborspa'].map({True:1,0:0})

df['calculatedvalue'] = df[['structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxamount']].sum(axis=1)



C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


We can drop 'finishedfloor1squarefeet','calculatedfinishedsquarefeet','finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50' and 'finishedsquarefeet6' because we marked the NaN values as 0, then took the max of all the columns and stored it in 'actual_area'

We can also drop: 
'bathroomcnt', 'threequarterbathnbr', 'fullbathcnt' and 'threequarterbathnbr' because 'calculatedbathnbr' stores all this information combined in one column.


In [112]:
age = []
for i in df['yearbuilt']:
    age.append(int(2015-i))

df['age'] = age
df.drop(['yearbuilt'],axis=1,inplace=True)

In [113]:
before_drop = df.copy()
print df.shape
df.drop(['architecturalstyletypeid','basementsqft', 'buildingclasstypeid', 'decktypeid',
        'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'roomcnt', 'yardbuildingsqft26', 
         'numberofstories', 'fireplaceflag', 'fullbathcnt', 'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid', 
         'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt', 
         'taxamount', 'bathroomcnt'], axis=1, inplace=True)
df.drop(['finishedfloor1squarefeet','calculatedfinishedsquarefeet','finishedsquarefeet12', 'finishedsquarefeet13',
              'finishedsquarefeet15', 'finishedsquarefeet50', 'finishedsquarefeet6'],axis=1, inplace=True)
print df.shape

(2985217, 61)
(2985217, 32)


In [114]:
before_rogue = df.copy()

In [ ]:
f, ax = plt.subplots(figsize=(15,15))
temp = pd.merge(before_drop,train,on='parcelid')
hmap = sbn.heatmap(pd.read_csv('./data/merge_copy.csv').corr('pearson'),
                   annot=True, fmt=".3f", linewidths=.5, ax=ax)
plt.show()

In [ ]:
fgure = hmap.get_figure()
fgure.savefig('before_drop_correlation_with_logerror.pdf')

In [ ]:
temp.isnull().sum()

In [ ]:
#going rogue

# propertycountylandusecode has 240 categorical values, so not using it, 
# same goes for propertyzoningdesc with too many categorical values
#regionidneighborhood couldnt comprehend how to fill null values, so leaving it out

df.drop(['propertycountylandusecode', 'propertyzoningdesc', 'rawcensustractandblock', 'censustractandblock',
         'regionidneighborhood'],axis=1, inplace = True)

In [ ]:

merged = pd.merge(df,train,on = 'parcelid')

# merged.info(null_counts= True)  ##checked, there are 0 NaN values in the merged dataframe

In [ ]:
dates = []
month = []
for i in merged['transactiondate']:
    dates.append(get_diff_month(datetime.strptime(i,'%Y-%m-%d'),datetime(2015,1,1)))
    month.append(pd.to_datetime(i, format='%Y-%m-%d').month)

# print month

merged['transactiondate'] = dates
merged['month_of_year'] = month
print merged.shape

In [ ]:
# print before_rogue.shape, merged.shape

In [ ]:
# Draw a heatmap with the numeric values in each cell
f, ax = plt.subplots(figsize=(20, 20))
sbn.heatmap(merged.corr('pearson'), annot=True, fmt="f", linewidths=.5, ax=ax)
plt.show()

In [ ]:
# month_v_error = merge_copy[['month_of_year','logerror']].plot.bar(figsize= (12,7))
month_v_error =  merged[['month_of_year','logerror']].groupby(by='month_of_year').mean().plot.line(figsize= (7,7),
                                                                                                   ylim =[0.0065,0.0195])
# plt.title('Percentage of missing values vs. properties in data')
# plt.xlabel('Properties of data')
# plt.ylabel('Percentage of missing values')
plt.show()

When we plot the month of the year against the mean logerror, we see that the value of the bar graph reduces in the months of March-June before rising up again in July. However, this is not to be intepreted as a reduction in the error, it only means that the mean is lower, the logerror could be negative. We will have to plot the month against the absolute error to see if their model is more accurate during the specified months or does it just give negative error.

In [ ]:
month_v_error = merged[['month_of_year','logerror']]
month_v_error['logerror'] = abs(month_v_error['logerror'])
month_v_error1 =  month_v_error[['month_of_year','logerror']].groupby(by='month_of_year').mean().plot.line(figsize= (7,7),
                                                                                                           ylim = [0.0065,0.08])
plt.show()

As suspected, there is no correlation between the month of the year and the absolute value of logerror. We can still use the value of 'month_of_year' as a feature and see if it benefits the model in anyway.

In [ ]:
age_v_error = merged[['age','logerror']].copy()
age_v_error['age'] = [2015-i for i in age_v_error['age']]
age_v_error.loc[:,'logerror'] = abs(month_v_error['logerror'])
# age_v_error = age_v_error[age_v_error['logerror'] <  age_v_error['logerror'].quantile(0.995)]  # exclude outliers
# age_v_error = age_v_error[age_v_error['logerror'] >  age_v_error['logerror'].quantile(0.005)]
age_v_error1 =  age_v_error[['age','logerror']].groupby(by='age').mean().plot.line(figsize= (17,7))
plt.xlabel('Year Built')
plt.ylabel('Log Error')
plt.show()

We can see that the mean absolute log error shows a reducing trend for properties that were more recently built when compared to the other older ones

In [ ]:
print merged.keys()

In [ ]:
# print merged['actual_area'].isnull().sum() #here we find out that actual area is null for 661 values, so we drop those points

# merged.dropna(axis=0,how='any',inplace=True)


Try to interpolate later if we have time

In [72]:
merged = pd.read_csv('./data/merge_copy.csv')

In [81]:
merged = pd.read_csv('./data/merge_copy_old.csv',index_col=0) 

In [ ]:
merged.keys(),merged.shape
# merged.to_csv('./data/merge_copy.csv',index=False)

In [ ]:
merge_copy = merged.copy()

In [ ]:
merged = merge_copy.copy() #shape should be (90275, 23)
# merged['logerror'] = logerror

In [ ]:
merged = merged[merged['logerror'] <  merged['logerror'].quantile(0.995)]  # exclude outliers
merged = merged[merged['logerror'] >  merged['logerror'].quantile(0.005)]

In [71]:
merged.keys()

Index([u'parcelid', u'airconditioningtypeid', u'bedroomcnt',
       u'buildingqualitytypeid', u'calculatedbathnbr', u'fips',
       u'fireplacecnt', u'garagetotalsqft', u'hashottuborspa', u'latitude',
       u'longitude', u'lotsizesquarefeet', u'poolcnt', u'regionidcounty',
       u'regionidzip', u'unitcnt', u'yardbuildingsqft17', u'actual_area',
       u'calculatedvalue', u'age', u'logerror', u'transactiondate',
       u'month_of_year'],
      dtype='object')

In [74]:
merged.keys()

Index([u'parcelid', u'airconditioningtypeid', u'bedroomcnt',
       u'buildingqualitytypeid', u'calculatedbathnbr', u'fips',
       u'fireplacecnt', u'garagecarcnt', u'garagetotalsqft', u'hashottuborspa',
       u'heatingorsystemtypeid', u'latitude', u'longitude',
       u'lotsizesquarefeet', u'poolcnt', u'propertylandusetypeid',
       u'regionidcity', u'regionidcounty', u'regionidzip', u'unitcnt',
       u'yardbuildingsqft17', u'taxdelinquencyflag', u'taxdelinquencyyear',
       u'blockgroup', u'actual_area', u'calculatedvalue', u'age', u'logerror',
       u'transactiondate', u'month_of_year'],
      dtype='object')

In [78]:
merged.drop(['garagecarcnt','heatingorsystemtypeid','propertylandusetypeid','regionidcity',u'taxdelinquencyflag',
             u'taxdelinquencyyear','blockgroup'],axis=1,inplace=True)

In [82]:
logerror = merged['logerror']

###merged[['bedroomcnt','calculatedbathnbr','fips','fireplacecnt',
#                                                           'garagetotalsqft', 'poolcnt','taxdelinquencyyear',
#                                                           'blockgroup','actual_area','age']]

X_train, X_test, y_train, y_test = train_test_split(merged.drop(['parcelid','logerror'],axis=1), logerror, test_size=0.35, random_state=42)

scaler = MinMaxScaler(feature_range=(-1,1)) 
s_train = scaler.fit_transform(X_train)
s_test = scaler.transform(X_test)

In [91]:
clf = LinearRegression(n_jobs=-1,normalize=False)
clf.fit(X_train,y_train)

res = clf.predict(X_test)
# res = [log(i,10) if i>0 else -log(i*-1,10) for i in res]
# print clf.score(X_test,y_test)
print mean_absolute_error(y_test,res) #,clf.score(X_test,y_test)
print mean_absolute_error(y_train,clf.predict(X_train))

0.0563530114422
0.0575027650224


WORK FOR NEXT TIME,TRY TO HANDLE NEGATIVE VALUES IN LOGERROR SO THAT AFTER TAKING ANTILOG, WE CAN AGAIN TAKE LOG FOR SUBMISSION PURPOSES

In [ ]:
print mean_squared_error(y_test,res)
# print mean_squared_error([log(i,10) if i>0 else -log(i*-1,10) for i in y_test],res)**0.5#,r2_score([log(i,10) for i in y_test],res)

In [ ]:
print mean_squared_error(y_test,res)**0.5

In [ ]:
# print merged.keys()
print pd.DataFrame({'columns': list(set(merged.keys()) - set(['logerror','parcelid'])),'coeffs': clf.coef_}) 

This information will be later used to find out the most important and the weakest attributes that are used for linear regression

The best feature so far in the model is the regionidzip with the coefficient -1.075408e-02 and the weakest feature are both  yardbuildingsqft17 and hashottuborspa with the coefficient -1.274611e+11 and 1.274611e+11 respectively

In [ ]:
# print res.mean(),y_test.mean()
print pd.DataFrame(res).describe(),"\n\n",pd.DataFrame(y_test).describe()

In [ ]:
plt.scatter(res,y_test)
plt.show()

In [96]:
lasso = linear_model.Lasso(alpha = 0.0001,tol=0.000000001,max_iter=2147483647,warm_start = False,normalize=False)
lasso.fit(X_train,y_train)
print mean_absolute_error(lasso.predict(X_test),y_test)

0.0563445175567


In [102]:
ridge = linear_model.Ridge(alpha = 3780,normalize=False, tol=0.000000001, max_iter=2147483647)
ridge.fit(X_train,y_train)
print mean_absolute_error(ridge.predict(X_test),y_test)

0.0563581708081


In [ ]:
####takes too long to train
# enet = linear_model.ElasticNet(alpha = 0.00008,normalize=False,l1_ratio=0.5,tol=0.000000001,max_iter=2147483647,warm_start=True)
# enet.fit(X_train,y_train)
# print mean_absolute_error(enet.predict(X_test),y_test)

In [61]:
##### Something is absurdly wrong with this classifier or the code


sgd = linear_model.SGDRegressor(penalty = 'l1',alpha=0.9, warm_start=True, max_iter=2147483647,tol=0.00000001)

sc = StandardScaler()
sgd_train = sc.fit_transform(X_train)
sgd_test = sc.transform(X_test)
sgd.fit(sgd_train,y_train)

print mean_absolute_error(sgd.predict(sgd_test),y_test)

# param_grid = {
#     'penalty': ('l1','l2'),
#     'alpha': [0.001,0.01,0.9],
#     'warm_start': [True,False],
#     'learning_rate' : ('optimal','invscaling')
#     }
    
# # start = datetime.now().time()
# gcv = GridSearchCV(sgd,param_grid,scoring = make_scorer(mean_squared_error,greater_is_better=False),n_jobs=4)
# gcv.fit(merged.drop(['parcelid','logerror'],axis=1),logerror)


# sgd_best = gcv.best_estimator_
# print gcv.best_score_
# print mean_absolute_error(sgd_best.predict(sgd_test),y_test)

0.0566458879163


In [ ]:
svr = svm.LinearSVR(C=0.1, max_iter=10000, tol=1e-06) # parameters not affecting the result
svr.fit(mlp_train,y_train)

In [109]:
print mean_absolute_error(svr.predict(mlp_test),y_test)

0.0561346662032


In [107]:
print "are you ready?"

are you ready?


C=10   0.057099199549
C=1    0.057099199549

MAE of default LinearSVR 0.0530913020565

In [ ]:
knr = KNR(n_jobs=-1)
knr_param_grid = {
    'n_neighbors' : [1800,2500,5000],
    'leaf_size': [200,300,600],
    'algorithm': ('auto', 'ball_tree', 'kd_tree', 'brute'),
    
    
}
knr_gcv = GridSearchCV(knr,knr_param_grid,scoring = make_scorer(mean_absolute_error,greater_is_better=False))

In [ ]:
knr_gcv.fit(s_train,y_train)
# print mean_absolute_error(knr.predict(X_test),y_test)

In [ ]:
best = knr_gcv.best_estimator_

In [ ]:
print mean_squared_error(best.predict(s_test),y_test), best

0.0568260301871 
KNeighborsRegressor(algorithm='auto', leaf_size=500, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=625, p=2,
          weights='uniform')
with scaled input 
0.0567777979104
same knr as above

using scaled only now
0.0567225844231 KNeighborsRegressor(algorithm='auto', leaf_size=300, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=1800, p=2,
          weights='uniform')




In [ ]:
svr = svm.LinearSVR(tol=0.0000000001,max_iter = 2147483647)
svr_param_grid = {
    'C' : [1,10,100,1000],
    'loss': ('epsilon_insensitive','squared_epsilon_insensitive')    
}

svr_gcv = GridSearchCV(svr,svr_param_grid,scoring = make_scorer(mean_absolute_error,greater_is_better=False),n_jobs=-1)

svr_gcv.fit(s_train,y_train)

In [ ]:
# print mean_absolute_error(svr.predict(s_test),y_test)

In [ ]:
merged.shape

In [128]:
sub = pd.read_csv('./data/sample_submission.csv')

sub.rename(columns={'ParcelId':'parcelid'},inplace=True)
sub_oct = pd.merge(df.drop(['garagecarcnt','heatingorsystemtypeid','propertylandusetypeid','regionidcity',u'taxdelinquencyflag',
                            'taxdelinquencyyear','blockgroup',u'propertycountylandusecode', u'propertyzoningdesc',
                            'rawcensustractandblock','censustractandblock','regionidneighborhood'],axis=1),sub,on='parcelid')

sub_oct.reset_index(inplace=True)
sub_oct.drop(['201610','201611','201612','201710','201711','201712','parcelid',"index"],axis=1,inplace=True)
# sub_oct.info(null_counts=True)


sub_new_dec = sub_oct.copy()
sub_new_nov = sub_oct.copy()
sub_new_oct = sub_oct.copy()
sub_dec = sub_oct.copy()
sub_nov = sub_oct.copy()

sub_oct['transactiondate'] = [22 for i in range(len(sub_oct))]
sub_oct['month_of_year'] = [10 for i in range(len(sub_oct))]
sub_oct = sc.transform(sub_oct)

sub_nov['transactiondate'] = [23 for i in range(len(sub_oct))]
sub_nov['month_of_year'] = [11 for i in range(len(sub_oct))]
sub_nov = sc.transform(sub_nov)

sub_dec['transactiondate'] = [24 for i in range(len(sub_oct))]
sub_dec['month_of_year'] = [12 for i in range(len(sub_oct))]
sub_dec = sc.transform(sub_dec)

sub_new_oct['transactiondate'] = [34 for i in range(len(sub_oct))]
sub_new_oct['month_of_year'] = [10 for i in range(len(sub_oct))]
sub_new_oct = sc.transform(sub_new_oct)

sub_new_nov['transactiondate'] = [35 for i in range(len(sub_oct))]
sub_new_nov['month_of_year'] = [11 for i in range(len(sub_oct))]
sub_new_nov = sc.transform(sub_new_nov)


sub_new_dec['transactiondate'] = [36 for i in range(len(sub_oct))]
sub_new_dec['month_of_year'] = [12 for i in range(len(sub_oct))]
sub_new_dec = sc.transform(sub_new_dec)

In [ ]:
print X_train.shape #,"\n\n",merged.keys()

In [129]:
res = sub.copy()
res['201610'] = svr.predict(sub_oct)
res['201611'] = svr.predict(sub_nov)
res['201612'] = svr.predict(sub_dec)
res['201710'] = svr.predict(sub_new_oct)
res['201711'] = svr.predict(sub_new_nov)
res['201712'] = svr.predict(sub_new_dec)

res.rename(columns={'parcelid':'ParcelId'},inplace=True)
res.to_csv('submission_svr.csv',index=False)

In [63]:
X_train['transactiondate'].describe()

count    57655.000000
mean        16.865718
std          2.815361
min         12.000000
25%         15.000000
50%         17.000000
75%         19.000000
max         23.000000
Name: transactiondate, dtype: float64

In [105]:
from sklearn.neural_network import MLPRegressor

sc = StandardScaler()
mlp_train = sc.fit_transform(X_train)
mlp_test = sc.transform(X_test)

mlp = MLPRegressor(learning_rate_init=0.9, activation='tanh',hidden_layer_sizes=(50,),learning_rate='invscaling',
                   solver='lbfgs',momentum=0.6, tol=0.00001, max_iter = 100000)


mlp.fit(mlp_train,y_train)

print mean_absolute_error(mlp.predict(mlp_test),y_test), mean_absolute_error(mlp.predict(mlp_train),y_train)

0.0584837499557 0.0574752816549


In [132]:
lr_train, lr_test, lr_y_train, lr_y_test = train_test_split(merged['actual_area'].reshape(-1, 1), logerror, test_size=0.35, random_state=42)


lr = LinearRegression(n_jobs=-1)

lr.fit(lr_train,lr_y_train)

pred = lr.predict(lr_test)
# res = [log(i,10) if i>0 else -log(i*-1,10) for i in res]
print mean_absolute_error(lr_y_test,pred) #, mean_squared_error(base_y_test,pred),base_clf.score(base_X_test,base_y_test)

0.0564229024614


C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.
